In [13]:
# Comandos mágicos para recargar cambios en tu script sin reiniciar el kernel
%load_ext autoreload
%autoreload 2

import sys
import os
import polars as pl


# ".." significa subir un nivel (salir de notebooks) y entrar a scripts
ruta_scripts = r'C:\Users\thoma\Documentos\Analisis de Datos\Portafolio Proyectos\Expo e Impo Colombia\src'
if ruta_scripts not in sys.path:
    sys.path.append(ruta_scripts)

# Ahora importamos tu función
try:
    import etl_tools as etl
    print("Librería etl_tools importada correctamente.")
except ImportError as e:
    print(f"Error: No se encuentra el archivo etl_tools.py en {ruta_scripts}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Error: No se encuentra el archivo etl_tools.py en C:\Users\thoma\Documentos\Analisis de Datos\Portafolio Proyectos\Expo e Impo Colombia\src


In [ ]:
#---IMPORTACIONES ----- 

#Rutas de entrada
ruta_zips_impo = os.path.join('..', 'Data', 'Impo')
ruta_zips_expo = os.path.join('..', 'Data', 'Expo')

# Rutas de salida 
lake_impo = os.path.join('..','Data', 'datalake_impo')
lake_expo = os.path.join('..', 'Data', 'datalake_expo')

#----Columnas que usaremos de importaciones------
cols_impo = ['FECH', 'PAISGEN' , 'VAFODO', 'PBK', 'ADUA', 'NABAN', 'REGIMEN']
txt_impo  = ['FECH', 'PAISGEN', 'ADUA', 'NABAN', 'REGIMEN']
num_impo  = ['VAFODO', 'PBK']

#----Columnas que usaremos de exportaciones------
cols_expo = ['FECH', 'PAIS' , 'FOBDOL', 'PBK', 'ADUA', 'POSAR', 'MODAD'] 
txt_expo  = ['FECH', 'PAIS' , 'ADUA', 'POSAR' , 'MODAD']
num_expo  = ['FOBDOL', 'PBK']


print("Configuración lista.")

In [11]:
print("🚀 INICIANDO CONSTRUCCIÓN DE DATA LAKES")
print("="*40)

# 1. Procesar IMPORTACIONES
print(f"\n📦 Procesando Importaciones...")
etl.zips_a_parquet(
    ruta_origen=ruta_zips_impo,
    ruta_destino=lake_impo,
    columnas_necesarias=cols_impo,
    cols_texto=txt_impo,
    cols_numero=num_impo
)

print("-" * 40)

# 2. Procesar EXPORTACIONES
print(f"\n📦 Procesando Exportaciones...")

try:
    etl.zips_a_parquet(
        ruta_origen=ruta_zips_expo,
        ruta_destino=lake_expo,
        columnas_necesarias=cols_expo,
        cols_texto=txt_expo,
        cols_numero=num_expo
    )
except Exception as e:
    print(f"⚠️ Error en Exportaciones: {e}")
    print("Consejo: Abre un archivo .dta de expo individualmente para verificar los nombres exactos de las columnas.")

print("\n✨ ¡PROCESO FINALIZADO!")

🚀 INICIANDO CONSTRUCCIÓN DE DATA LAKES

📦 Procesando Importaciones...


NameError: name 'etl' is not defined

In [ ]:
# Verificación rápida
print("--- Auditoría de Datos ---")

try:
    # Chequeo Impo
    df_impo = pl.scan_parquet(os.path.join(lake_impo, "*.parquet"))
    conteo_impo = df_impo.select(pl.len()).collect().item()
    print(f"✅ Data Lake Importaciones: {conteo_impo:,.0f} registros accesibles.")
    
    # Chequeo Expo
    df_expo = pl.scan_parquet(os.path.join(lake_expo, "*.parquet"))
    conteo_expo = df_expo.select(pl.len()).collect().item()
    print(f"✅ Data Lake Exportaciones: {conteo_expo:,.0f} registros accesibles.")

except Exception as e:
    print(f"❌ Algo salió mal al leer los parquets: {e}")